In [2]:
from ultralytics import YOLO
import os 

In [3]:
def run_inference(weights_path, source, img_size=640, device='0'):
	"""
	Function to run inference using the trained YOLOv11 model.

	Parameters:
	- weights_path: Path to the trained weights
	- source: Directory or image path to run inference on
	- img_size: Image size for inference (default: 640)
	- conf_thresh: Confidence threshold (default: 0.25)
	- device: Device to run on ('0' for GPU, 'cpu' for CPU)
	"""
	
	print(f"Running inference on images in {source}")
	model = YOLO(weights_path).to(device)
	image_filenames = [filename for filename in os.listdir(source) if filename.endswith(('.jpg', '.jpeg', '.png'))]
	image_paths = [os.path.join(source, filename) for filename in image_filenames]
	results = [model(img_path, imgsz=img_size) for img_path in image_paths]
	print("Inference Complete.")
	return results

In [4]:
weights_path = "C:/Users/Usuario/Documents/GitHub/ANPR-GIA/yolo11n_licenseplates.pt"
source = "C:/Users/Usuario/Documents/GitHub/ANPR-GIA/images/reduced"
img_size = 640 
device = "cpu"

In [5]:
results = run_inference(weights_path,source,img_size, device = device)

Running inference on images in C:/Users/Usuario/Documents/GitHub/ANPR-GIA/images/reduced

image 1/1 C:\Users\Usuario\Documents\GitHub\ANPR-GIA\images\reduced\0.jpg: 384x640 1 license_plate, 88.9ms
Speed: 3.1ms preprocess, 88.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\Usuario\Documents\GitHub\ANPR-GIA\images\reduced\1.jpg: 384x640 1 license_plate, 71.2ms
Speed: 2.5ms preprocess, 71.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\Usuario\Documents\GitHub\ANPR-GIA\images\reduced\10.jpeg: 480x640 1 license_plate, 100.2ms
Speed: 3.1ms preprocess, 100.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\Usuario\Documents\GitHub\ANPR-GIA\images\reduced\11.jpeg: 640x480 1 license_plate, 91.4ms
Speed: 2.5ms preprocess, 91.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 C:\Users\Usuario\Documents\GitHub\ANPR-GIA\images\reduced\2.jpg: 288x640 1

In [6]:
results

[[ultralytics.engine.results.Results object with attributes:
  
  boxes: ultralytics.engine.results.Boxes object
  keypoints: None
  masks: None
  names: {0: 'license_plate'}
  obb: None
  orig_img: array([[[255, 254, 253],
          [255, 254, 253],
          [255, 254, 253],
          ...,
          [255, 251, 242],
          [255, 251, 242],
          [255, 252, 243]],
  
         [[255, 254, 253],
          [255, 254, 253],
          [255, 254, 253],
          ...,
          [255, 250, 241],
          [254, 249, 240],
          [254, 249, 240]],
  
         [[252, 250, 249],
          [252, 250, 249],
          [252, 250, 249],
          ...,
          [250, 248, 240],
          [250, 248, 240],
          [250, 248, 240]],
  
         ...,
  
         [[133, 138, 147],
          [130, 135, 144],
          [131, 136, 145],
          ...,
          [113, 136, 144],
          [115, 138, 146],
          [113, 136, 144]],
  
         [[134, 139, 148],
          [131, 136, 145],
        

In [7]:
results[0][0].boxes.xyxy[0].cpu().numpy().astype(int).tolist()

[1237, 1200, 2557, 1568]

In [10]:
import cv2
import matplotlib.pyplot as plt
import math

# Determine the number of rows and columns for the subplot grid
num_images = 4
cols = 3  # Number of columns (adjust as needed)
rows = math.ceil(num_images / cols)  # Calculate number of rows

# Create subplots
fig, axes = plt.subplots(rows, cols, figsize=(15, 15))

# Flatten the axes array for easy indexing
axes = axes.flatten()

# Loop over the images and display them in subplots
for idx, result in enumerate(results):
	result = result[0]
	img = result.orig_img
	box = result.boxes.xyxy[0].cpu().numpy().astype(int).tolist()
	if img is not None:
		img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

		# Draw the bounding box on the image
		x1, y1, x2, y2 = box
		cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

		# Display the image in the subplot
		axes[idx].imshow(img)
		axes[idx].axis('off')  # Hide the axis
	else:
		print("Could not load image")

# Remove any unused subplots
for i in range(num_images, len(axes)):
	fig.delaxes(axes[i])

# Display the grid of images
plt.tight_layout()
plt.show()

IndexError: index 6 is out of bounds for axis 0 with size 6

: 

In [ ]:
from PIL import Image
import cv2

example_img = cv2.imread('../images/diverse/Copia de 1727095338102.jpg')
example_img = cv2.cvtColor(example_img, cv2.COLOR_BGR2RGB)
example_img = Image.fromarray(example_img)
lst = [example_img]*3
model = YOLO('../yolo11n_finetuned.pt')

In [ ]:
results = model("../images/reduced", imgsz=640, verbose=False)
box = results[0][0].boxes.xyxy[0].cpu().numpy().astype(int).tolist()
crop = Image.fromarray(results[0].orig_img[:,:,::-1]).crop(box)
crop